In [12]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [4]:
def sample_pixels(images, num_samples):
    
    # Choose pixels with largest standard deviation
    stds = np.std(images, axis=0)
    selected_pixel_value = np.argsort(stds.ravel())[::-1][:num_samples]
    sampled_pixels = np.unravel_index(selected_pixel_value, stds.shape) 
    # returns (i of chosen pixels, j of chosen pixels)
    
    # Compute the sample values for each pixel
    samples = np.zeros((num_samples, len(images)), dtype=np.float32)
    for i in range(num_samples):
        x, y = sampled_pixels[0][i], sampled_pixels[1][i]
        for j in range(len(images)):
            samples[i, j] = images[j][x, y]
    
    return samples
    # returns [ [first chosen pixel in every image], [], ..... [] ]

In [10]:
def solve_respond_curve(z,lgT,lam,w):
    #z : ith pixel of those chosen in jth image
    n = len(z) #n : we choose n pixels total for each images
    p = len(z[0])#p : we choose p images total 
    row = n*p + 1 + 254
    col = 256 + n   # row and col in matrix a
    a = [[] for i in range(row)]
    b = [[] for i in range(row)]

    #initialize matrix a(zero) and b(zero)
    for i in range(row):
        b[i].append(0)
        for j in range(col):
            a[i].append(0)

    #upper half of matrix a and b done
    for j in range(p):
        for i in range(n):
            Z = int(z[i][j])
            b[i+j*n][0] = lgT[j] * w[Z]
            for r in range(256):
                if r == Z :
                    a[i+j*n][r] = 1 * w[Z]
            a[i+j*n][i] = -1 * w[Z]

    #fix the curve by setting its middle value to 0
    a[n*p][128] = 1

    #lower half of matrix a and b done
    for k in range(n*p+1,n*p+255):
        for l in range(255):
            a[k][l] = 1 * w[l+1] * lam 
            a[k][l+1] = -2 * w[l+1] * lam 
            a[k][l+2] = 1 * w[l+1] * lam 
            
    
    A = np.array(a)
    B = np.array(b)

    print(np.shape(A))
    print(np.shape(B))
    
    #Solve the system using SVD
    x = np.linalg.lstsq(A,B)[0]
    g = x[:256]
    lE = x[256:]

    return g, lE

In [11]:
if __name__ == '__main__':
    folder = "testing_image" # for now
    images = np.array(load_images(folder))
    num_samples = 50 # for now

    z0 = sample_pixels(images[:,:,:,0], num_samples)
    z1 = sample_pixels(images[:,:,:,1], num_samples)
    z2 = sample_pixels(images[:,:,:,2], num_samples)
    
    exposure_times = [1/4000,1/2000,1/1000,1/500,1/250,1/125,1/60,1/30,1/15,1/8,1/4,1/2,1] # for now
    lambda_ = 50 # for now
    w = [value if value <= 0.5*255 else 255-value for value in range(256)]
    
    g, lnE = solve_respond_curve(z0, exposure_times, lambda_, w)
    
    
    

[[  3. 220. 254. 254. 255.   5. 251. 254.   0. 253.   0.   1. 226.]
 [ 14. 221. 254. 254. 255.   7. 251. 254.   0. 253.   1.   1. 231.]
 [ 15. 220. 254. 254. 255.   5. 255. 255.   0. 255.   0.   5. 225.]
 [ 22. 224. 254. 254. 255.   0. 251. 254.   5. 251.   0.   2. 230.]
 [  0. 177. 254. 254. 255.   0. 255. 254.   0. 252.   0.   0. 192.]
 [ 28. 238. 255. 254. 255.   6. 255. 255.  10. 255.   1.   6. 243.]
 [  5. 188. 254. 254. 255.   6. 254. 254.   3. 255.   4.   2. 216.]
 [  0. 156. 254. 254. 255.   0. 255. 254.   0. 252.   0.   0. 176.]
 [  0.  33. 254. 254. 255.   6. 239. 254.   3. 220.   4.   2.  53.]
 [ 11. 198. 254. 254. 255.   1. 254. 252.   1. 251.   2.   2. 203.]
 [  0.  31. 254. 254. 255.   8. 235. 254.   4. 220.   8.   2.  40.]
 [  8. 209. 255. 254. 255.   9. 250. 254.   7. 251.   6.   2. 228.]
 [  0.  36. 254. 254. 255.   5. 236. 254.   3. 223.   6.   2.  50.]
 [ 10. 190. 254. 254. 255.   5. 251. 254.   0. 251.   0.   0. 201.]
 [  0. 182. 254. 254. 255.  11. 255. 254.   0. 2

/var/folders/bz/qpg9g_nj7lx_0r49_7jyc15c0000gp/T/ipykernel_47174/495197369.py:44: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x = np.linalg.lstsq(A,B)[0]
